# 📈 Data Analytics & Prognose in der Produktionsplanung

**Intelligente Produktionsplanung und KI**  
Universität Stuttgart  
Master Maschinenbau / Technologiemanagement

---

## Lernziele

Nach Bearbeitung dieses Notebooks können Sie:

1. **Zeitreihen analysieren** und in Komponenten (Trend, Saisonalität, Rest) zerlegen
2. Verschiedene **Prognosemethoden** anwenden (Moving Average, Exponentielle Glättung, Regression)
3. Die **Prognosegenauigkeit** mit MAE, RMSE und MAPE bewerten
4. **Saisonale Muster** erkennen und in Prognosen integrieren
5. **Externe Faktoren** (Promotions, Feiertage) in Prognosemodelle einbeziehen

---

## 1. Einführung und Grundlagen

### Warum Prognosen in der Produktionsplanung?

Genaue Nachfrageprognosen sind **essentiell** für:

| Bereich | Auswirkung guter Prognosen |
|---------|---------------------------|
| **Bestandsmanagement** | Optimale Lagerbestände, weniger Kapitalbindung |
| **Kapazitätsplanung** | Vermeidung von Über-/Unterkapazitäten |
| **Beschaffung** | Rechtzeitige Materialbestellung, bessere Konditionen |
| **Personalplanung** | Bedarfsgerechter Personaleinsatz |

### Komponenten einer Zeitreihe

Eine Zeitreihe $Y_t$ kann in folgende Komponenten zerlegt werden:

$$Y_t = T_t + S_t + C_t + \epsilon_t$$

| Komponente | Symbol | Beschreibung |
|------------|--------|-------------|
| **Trend** | $T_t$ | Langfristige Auf- oder Abwärtsbewegung |
| **Saisonalität** | $S_t$ | Regelmäßige, sich wiederholende Muster (z.B. jährlich) |
| **Zyklus** | $C_t$ | Längerfristige Schwankungen (Konjunktur) |
| **Restkomponente** | $\epsilon_t$ | Zufällige Schwankungen (Rauschen) |

In [ ]:
# ===================================================================
# Imports und Konfiguration
# ===================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Matplotlib Konfiguration
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 11
plt.style.use('seaborn-v0_8-whitegrid')

print("✅ Alle Bibliotheken erfolgreich geladen!")
print("\n📦 Verwendete Pakete:")
print(f"   - NumPy: {np.__version__}")
print(f"   - Pandas: {pd.__version__}")

---

## 2. Fallbeispiel: Nachfrageprognose bei ElektroTech GmbH

### Problemstellung

Die **ElektroTech GmbH** ist ein Elektronikhersteller mit vier Produktgruppen:
- Smartphones
- Tablets
- Laptops
- Zubehör

Das Unternehmen möchte die **monatliche Nachfrage** für die kommenden 6 Monate prognostizieren, um:
- Produktionskapazitäten zu planen
- Materialbestellungen zu optimieren
- Lagerbestände zu minimieren

### Datengenerierung

Wir generieren **realistische Nachfragedaten** mit:
- **Trend**: Langfristige Wachstumsraten
- **Saisonalität**: Höhere Nachfrage im Q4 (Weihnachtsgeschäft)
- **Black Friday Effekt**: Nachfragespitze im November
- **Zufällige Schwankungen**: Unvorhersehbare Faktoren

In [ ]:
# ===================================================================
# Datensatz generieren: Nachfragedaten
# ===================================================================

def generiere_nachfragedaten(perioden=36, seed=42):
    """
    Generiert realistische Nachfragedaten für verschiedene Produktgruppen
    mit Trend, Saisonalität und Zufallskomponente.
    
    Parameter:
    ----------
    perioden : int
        Anzahl der Monate
    seed : int
        Seed für Reproduzierbarkeit
        
    Returns:
    --------
    DataFrame mit Nachfragedaten pro Produktgruppe
    """
    np.random.seed(seed)
    
    # Zeitindex (Monate)
    datum = pd.date_range(start='2021-01-01', periods=perioden, freq='M')
    
    # Produktparameter
    produkte = {
        'Smartphones': {'basis': 1000, 'trend': 0.02, 'saison_amp': 150, 'rauschen': 0.08},
        'Tablets':     {'basis': 500,  'trend': 0.01, 'saison_amp': 80,  'rauschen': 0.10},
        'Laptops':     {'basis': 350,  'trend': 0.015,'saison_amp': 60,  'rauschen': 0.12},
        'Zubehoer':    {'basis': 800,  'trend': 0.03, 'saison_amp': 120, 'rauschen': 0.07}
    }
    
    daten = {}
    
    for produkt, params in produkte.items():
        basis = params['basis']
        
        # 1. Trend-Komponente (exponentielles Wachstum)
        trend = basis * (1 + params['trend']) ** np.arange(perioden)
        
        # 2. Saisonale Komponente (Peak im Q4, Tief im Q1)
        monat = np.arange(perioden) % 12 + 1
        saison = params['saison_amp'] * np.sin(2 * np.pi * (monat - 3) / 12)
        
        # 3. Black Friday Effekt (November = Monat 11)
        black_friday = np.where(monat == 11, basis * 0.25, 0)
        
        # 4. Zufällige Schwankungen
        rauschen = np.random.normal(0, basis * params['rauschen'], perioden)
        
        # Gesamtnachfrage
        nachfrage = trend + saison + black_friday + rauschen
        nachfrage = np.maximum(nachfrage, basis * 0.5)  # Minimum 50% der Basis
        
        daten[produkt] = np.round(nachfrage, 0)
    
    df = pd.DataFrame(daten, index=datum)
    df.index.name = 'Datum'
    
    return df

# Daten generieren
nachfrage_df = generiere_nachfragedaten(perioden=36)

print("📊 GENERIERTE NACHFRAGEDATEN")
print("=" * 65)
print(f"\nZeitraum: {nachfrage_df.index[0].strftime('%Y-%m')} bis {nachfrage_df.index[-1].strftime('%Y-%m')}")
print(f"Anzahl Perioden: {len(nachfrage_df)} Monate")
print(f"Produkte: {list(nachfrage_df.columns)}")

print(f"\n📋 Erste 6 Datenpunkte:")
print(nachfrage_df.head(6).to_string())

print(f"\n📈 Statistische Übersicht:")
print(nachfrage_df.describe().round(0).to_string())

In [ ]:
# ===================================================================
# Visualisierung der Nachfragedaten
# ===================================================================

def plot_nachfragedaten(df):
    """
    Visualisiert die Nachfragedaten aller Produktgruppen.
    """
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    farben = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D']
    
    for i, (produkt, farbe) in enumerate(zip(df.columns, farben)):
        ax = axes[i // 2, i % 2]
        ax.plot(df.index, df[produkt], marker='o', markersize=4, 
               linewidth=1.5, color=farbe, label=produkt)
        
        # Gleitender Durchschnitt als Trend
        ma = df[produkt].rolling(window=3).mean()
        ax.plot(df.index, ma, '--', color='gray', linewidth=2, 
               alpha=0.7, label='3-Monats-Durchschnitt')
        
        ax.set_title(f'Nachfrage: {produkt}', fontsize=12, fontweight='bold')
        ax.set_xlabel('Datum')
        ax.set_ylabel('Nachfrage (Stück)')
        ax.legend(loc='upper left')
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis='x', rotation=45)
        
        # November (Black Friday) markieren
        for date in df.index:
            if date.month == 11:
                ax.axvline(x=date, color='red', linestyle=':', alpha=0.3)
    
    plt.suptitle('Nachfrageentwicklung ElektroTech GmbH (2021-2023)', 
                fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    
    # Gesamtübersicht
    fig, ax = plt.subplots(figsize=(14, 6))
    
    for produkt, farbe in zip(df.columns, farben):
        ax.plot(df.index, df[produkt], marker='o', markersize=4,
               linewidth=2, label=produkt, color=farbe)
    
    ax.set_title('Nachfragevergleich aller Produktgruppen', fontsize=12, fontweight='bold')
    ax.set_xlabel('Datum')
    ax.set_ylabel('Nachfrage (Stück)')
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

print("📊 VISUALISIERUNG DER NACHFRAGEDATEN")
print("=" * 50)
print("(Rote gestrichelte Linien markieren November = Black Friday)")
plot_nachfragedaten(nachfrage_df)

---

## 3. Zeitreihenzerlegung

### Additive vs. Multiplikative Zerlegung

| Modell | Formel | Anwendung |
|--------|--------|-----------|
| **Additiv** | $Y_t = T_t + S_t + \epsilon_t$ | Saisonale Schwankungen konstant |
| **Multiplikativ** | $Y_t = T_t \times S_t \times \epsilon_t$ | Saisonale Schwankungen proportional zum Niveau |

### Methoden zur Trendschätzung

1. **Gleitender Durchschnitt**: Glättet kurzfristige Schwankungen
2. **Lineare Regression**: Schätzt linearen Trend
3. **Polynomiale Regression**: Für nichtlineare Trends

In [ ]:
# ===================================================================
# Zeitreihenzerlegung
# ===================================================================

def zeitreihen_zerlegung(serie, produkt_name):
    """
    Zerlegt eine Zeitreihe in Trend, Saisonalität und Restkomponente.
    
    Parameter:
    ----------
    serie : pd.Series
        Die zu zerlegende Zeitreihe
    produkt_name : str
        Name für die Visualisierung
        
    Returns:
    --------
    tuple : (trend, saisonalitaet, rest)
    """
    
    # 1. Trend: 12-Monats gleitender Durchschnitt (zentriert)
    trend = serie.rolling(window=12, center=True).mean()
    
    # 2. Detrended Serie
    detrended = serie - trend
    
    # 3. Saisonale Komponente: Durchschnitt pro Monat
    saisonale_indizes = detrended.groupby(detrended.index.month).mean()
    # Normalisieren (Summe = 0 für additives Modell)
    saisonale_indizes = saisonale_indizes - saisonale_indizes.mean()
    
    # Saisonalität auf volle Serie anwenden
    saisonalitaet = serie.index.month.map(lambda m: saisonale_indizes[m])
    saisonalitaet = pd.Series(saisonalitaet.values, index=serie.index)
    
    # 4. Rest-Komponente
    rest = serie - trend - saisonalitaet
    
    # Visualisierung
    fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)
    
    # Original
    axes[0].plot(serie.index, serie, 'b-', linewidth=1.5)
    axes[0].set_title(f'{produkt_name}: Originaldaten', fontweight='bold')
    axes[0].set_ylabel('Nachfrage')
    axes[0].grid(True, alpha=0.3)
    
    # Trend
    axes[1].plot(trend.index, trend, 'r-', linewidth=2)
    axes[1].set_title('Trend-Komponente (12-Monats gleitender Durchschnitt)', fontweight='bold')
    axes[1].set_ylabel('Trend')
    axes[1].grid(True, alpha=0.3)
    
    # Saisonalität
    axes[2].plot(saisonalitaet.index, saisonalitaet, 'g-', linewidth=1.5)
    axes[2].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    axes[2].set_title('Saisonale Komponente', fontweight='bold')
    axes[2].set_ylabel('Saisonalität')
    axes[2].grid(True, alpha=0.3)
    
    # Rest
    axes[3].plot(rest.index, rest, 'orange', linewidth=1, alpha=0.8)
    axes[3].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    axes[3].fill_between(rest.index, rest, 0, alpha=0.3, color='orange')
    axes[3].set_title('Rest-Komponente (Rauschen)', fontweight='bold')
    axes[3].set_ylabel('Rest')
    axes[3].set_xlabel('Datum')
    axes[3].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Saisonale Indizes anzeigen
    print(f"\n📅 SAISONALE INDIZES für {produkt_name}:")
    print("-" * 50)
    monate = ['Jan', 'Feb', 'Mär', 'Apr', 'Mai', 'Jun', 
             'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dez']
    for i, (monat, wert) in enumerate(zip(monate, saisonale_indizes.values), 1):
        symbol = '📈' if wert > 0 else '📉'
        print(f"   {monat}: {wert:+7.1f} {symbol}")
    
    return trend, saisonalitaet, rest

# Zerlegung für Smartphones
print("📊 ZEITREIHENZERLEGUNG")
print("=" * 50)
trend, saison, rest = zeitreihen_zerlegung(nachfrage_df['Smartphones'], 'Smartphones')

print("\n💡 INTERPRETATION:")
print("   • Der Trend zeigt stetiges Wachstum")
print("   • November hat den höchsten saisonalen Index (Black Friday)")
print("   • Q1 zeigt niedrigere Nachfrage (nach Weihnachten)")

---

## 4. Prognosemethoden

### Übersicht der Methoden

| Methode | Beschreibung | Vorteile | Nachteile |
|---------|-------------|----------|----------|
| **Naiv** | Letzter Wert als Prognose | Einfach, Benchmark | Ignoriert Muster |
| **Moving Average** | Durchschnitt der letzten n Werte | Glättet Rauschen | Verzögert bei Trends |
| **Exp. Glättung** | Gewichteter Durchschnitt (neuere Werte stärker) | Flexibel, wenig Daten | Nur für stationäre Reihen |
| **Holt** | Exp. Glättung + Trend | Berücksichtigt Trend | Keine Saisonalität |
| **Holt-Winters** | Holt + Saisonalität | Vollständig | Viele Parameter |
| **Regression** | Lineare/polynomiale Modelle | Interpretierbar | Nur linearer Trend |

### Genauigkeitsmaße

| Maß | Formel | Interpretation |
|----|--------|---------------|
| **MAE** | $\frac{1}{n}\sum|y_i - \hat{y}_i|$ | Mittlerer absoluter Fehler (gleiche Einheit) |
| **RMSE** | $\sqrt{\frac{1}{n}\sum(y_i - \hat{y}_i)^2}$ | Bestraft große Fehler stärker |
| **MAPE** | $\frac{100}{n}\sum\frac{|y_i - \hat{y}_i|}{|y_i|}$ | Prozentualer Fehler (vergleichbar) |

In [ ]:
# ===================================================================
# Prognosemethoden Implementation
# ===================================================================

class PrognoseMethoden:
    """
    Sammlung verschiedener Prognosemethoden für Zeitreihen.
    """
    
    @staticmethod
    def naive(serie, h=1):
        """
        Naive Prognose: Letzter Wert wird fortgeschrieben.
        
        Parameter:
        ----------
        serie : pd.Series
            Historische Daten
        h : int
            Prognosehorizont
            
        Returns:
        --------
        list : Prognosen für h Perioden
        """
        return [serie.iloc[-1]] * h
    
    @staticmethod
    def saisonale_naive(serie, saison_laenge=12, h=1):
        """
        Saisonale naive Prognose: Wert der gleichen Saison im Vorjahr.
        """
        prognosen = []
        for i in range(h):
            idx = -(saison_laenge - i % saison_laenge)
            prognosen.append(serie.iloc[idx])
        return prognosen
    
    @staticmethod
    def moving_average(serie, window=3, h=1):
        """
        Gleitender Durchschnitt.
        
        Parameter:
        ----------
        serie : pd.Series
            Historische Daten
        window : int
            Fensterbreite
        h : int
            Prognosehorizont
            
        Returns:
        --------
        list : Prognosen für h Perioden
        """
        ma = serie.iloc[-window:].mean()
        return [ma] * h
    
    @staticmethod
    def exponential_smoothing(serie, alpha=0.3, h=1):
        """
        Einfache exponentielle Glättung (Simple Exponential Smoothing).
        
        Parameter:
        ----------
        serie : pd.Series
            Historische Daten
        alpha : float
            Glättungsparameter (0 < alpha < 1)
            Höheres alpha = stärkere Gewichtung aktueller Werte
        h : int
            Prognosehorizont
            
        Returns:
        --------
        list : Prognosen für h Perioden
        """
        # Initialisierung mit erstem Wert
        level = serie.iloc[0]
        
        # Durch alle historischen Daten iterieren
        for t in range(1, len(serie)):
            level = alpha * serie.iloc[t] + (1 - alpha) * level
        
        return [level] * h
    
    @staticmethod
    def holt_linear(serie, alpha=0.3, beta=0.1, h=1):
        """
        Holt's lineare Methode (Double Exponential Smoothing).
        Berücksichtigt Trend.
        
        Parameter:
        ----------
        serie : pd.Series
            Historische Daten
        alpha : float
            Level-Glättungsparameter
        beta : float
            Trend-Glättungsparameter
        h : int
            Prognosehorizont
            
        Returns:
        --------
        list : Prognosen für h Perioden
        """
        # Initialisierung
        level = serie.iloc[0]
        trend = serie.iloc[1] - serie.iloc[0]
        
        # Update für alle historischen Daten
        for t in range(1, len(serie)):
            level_prev = level
            level = alpha * serie.iloc[t] + (1 - alpha) * (level + trend)
            trend = beta * (level - level_prev) + (1 - beta) * trend
        
        # Prognosen
        prognosen = [level + (i + 1) * trend for i in range(h)]
        return prognosen
    
    @staticmethod
    def linear_regression(serie, h=1):
        """
        Lineare Regression für Trendextrapolation.
        
        Returns:
        --------
        list : Prognosen für h Perioden
        """
        X = np.arange(len(serie)).reshape(-1, 1)
        y = serie.values
        
        model = LinearRegression()
        model.fit(X, y)
        
        X_future = np.arange(len(serie), len(serie) + h).reshape(-1, 1)
        prognosen = model.predict(X_future)
        
        return list(prognosen)

# Instanz erstellen
prog = PrognoseMethoden()

print("✅ Prognosemethoden geladen:")
print("   • Naive Prognose")
print("   • Saisonale naive Prognose")
print("   • Moving Average")
print("   • Exponential Smoothing")
print("   • Holt's lineare Methode")
print("   • Lineare Regression")

In [ ]:
# ===================================================================
# Genauigkeitsmaße
# ===================================================================

def berechne_genauigkeit(y_true, y_pred):
    """
    Berechnet verschiedene Genauigkeitsmaße.
    
    Parameter:
    ----------
    y_true : array-like
        Tatsächliche Werte
    y_pred : array-like
        Prognostizierte Werte
        
    Returns:
    --------
    dict : Dictionary mit MAE, RMSE, MAPE
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    # MAPE (Vermeidung Division durch Null)
    mask = y_true != 0
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    
    return {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}

print("📊 ERKLÄRUNG DER GENAUIGKEITSMASSE")
print("=" * 60)
print("""
MAE (Mean Absolute Error):
   • Mittlerer absoluter Prognosefehler
   • Gleiche Einheit wie die Daten (z.B. Stück)
   • Interpretation: "Im Schnitt liegen wir X Einheiten daneben"

RMSE (Root Mean Squared Error):
   • Wurzel des mittleren quadratischen Fehlers
   • Bestraft große Fehler überproportional
   • RMSE ≥ MAE (gleich nur bei konstanten Fehlern)

MAPE (Mean Absolute Percentage Error):
   • Mittlerer prozentualer Fehler
   • Unabhängig von der Skala → vergleichbar zwischen Produkten
   • Faustregel: MAPE < 10% = sehr gut, < 20% = gut
""")

---

## 5. Prognose-Vergleich

Wir vergleichen die verschiedenen Prognosemethoden anhand eines **Train-Test-Splits**:
- **Training**: Erste 30 Monate (zur Modellbildung)
- **Test**: Letzte 6 Monate (zur Evaluation)

In [ ]:
# ===================================================================
# Prognose-Vergleich für ein Produkt
# ===================================================================

def vergleiche_prognosemethoden(serie, produkt_name, test_perioden=6):
    """
    Vergleicht verschiedene Prognosemethoden.
    
    Parameter:
    ----------
    serie : pd.Series
        Vollständige Zeitreihe
    produkt_name : str
        Name des Produkts
    test_perioden : int
        Anzahl der Testperioden
        
    Returns:
    --------
    DataFrame : Ergebnisse des Vergleichs
    """
    
    # Daten aufteilen
    train = serie[:-test_perioden]
    test = serie[-test_perioden:]
    
    print(f"\n📊 PROGNOSE-VERGLEICH: {produkt_name}")
    print("=" * 65)
    print(f"Training: {len(train)} Perioden ({train.index[0].strftime('%Y-%m')} bis {train.index[-1].strftime('%Y-%m')})")
    print(f"Test:     {len(test)} Perioden ({test.index[0].strftime('%Y-%m')} bis {test.index[-1].strftime('%Y-%m')})")
    
    # Prognosen erstellen
    methoden = {
        'Naive':              prog.naive(train, test_perioden),
        'Saisonale Naive':    prog.saisonale_naive(train, 12, test_perioden),
        'Moving Avg (3)':     prog.moving_average(train, 3, test_perioden),
        'Moving Avg (6)':     prog.moving_average(train, 6, test_perioden),
        'Exp. Smoothing':     prog.exponential_smoothing(train, 0.3, test_perioden),
        'Holt Linear':        prog.holt_linear(train, 0.3, 0.1, test_perioden),
        'Lin. Regression':    prog.linear_regression(train, test_perioden)
    }
    
    # Genauigkeit berechnen
    ergebnisse = []
    
    print(f"\n{'Methode':<20} {'MAE':>10} {'RMSE':>10} {'MAPE (%)':>10}")
    print("-" * 55)
    
    for methode, prognose in methoden.items():
        genauigkeit = berechne_genauigkeit(test.values, prognose)
        ergebnisse.append({
            'Methode': methode,
            'MAE': genauigkeit['MAE'],
            'RMSE': genauigkeit['RMSE'],
            'MAPE': genauigkeit['MAPE'],
            'Prognose': prognose
        })
        print(f"{methode:<20} {genauigkeit['MAE']:>10.1f} {genauigkeit['RMSE']:>10.1f} {genauigkeit['MAPE']:>10.1f}")
    
    # Beste Methode identifizieren
    df_ergebnisse = pd.DataFrame(ergebnisse)
    beste_idx = df_ergebnisse['MAE'].idxmin()
    beste_methode = df_ergebnisse.loc[beste_idx, 'Methode']
    beste_mae = df_ergebnisse.loc[beste_idx, 'MAE']
    
    print("-" * 55)
    print(f"🏆 BESTE METHODE: {beste_methode} (MAE: {beste_mae:.1f})")
    
    # Visualisierung
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Trainingsdaten
    ax.plot(train.index, train.values, 'b-', linewidth=2, label='Training', alpha=0.7)
    
    # Testdaten (tatsächlich)
    ax.plot(test.index, test.values, 'go-', linewidth=3, markersize=10, 
           label='Tatsächlich', zorder=5)
    
    # Prognosen
    farben = ['red', 'orange', 'purple', 'brown', 'pink', 'cyan', 'magenta']
    linestyles = ['--', '-.', ':', '--', '-.', ':', '--']
    
    for i, row in df_ergebnisse.iterrows():
        ax.plot(test.index, row['Prognose'], 
               linestyle=linestyles[i % len(linestyles)], 
               color=farben[i % len(farben)],
               linewidth=2, marker='s', markersize=6,
               label=f"{row['Methode']} (MAE: {row['MAE']:.0f})")
    
    ax.set_title(f'Prognose-Vergleich: {produkt_name}', fontsize=14, fontweight='bold')
    ax.set_xlabel('Datum', fontsize=11)
    ax.set_ylabel('Nachfrage (Stück)', fontsize=11)
    ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=9)
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)
    
    # Vertikale Linie zwischen Train und Test
    ax.axvline(x=train.index[-1], color='gray', linestyle='--', linewidth=2, alpha=0.5)
    ax.text(train.index[-1], ax.get_ylim()[1], ' Train | Test ', 
           fontsize=10, ha='center', va='bottom', color='gray')
    
    plt.tight_layout()
    plt.show()
    
    return df_ergebnisse

# Vergleich für Smartphones
ergebnisse_smartphones = vergleiche_prognosemethoden(
    nachfrage_df['Smartphones'], 'Smartphones', test_perioden=6
)

---

## 6. Saisonalitätsanalyse

Die Saisonalität ist ein wichtiger Faktor in der Nachfrageprognose. Wir analysieren die saisonalen Muster über alle Produktgruppen.

In [ ]:
# ===================================================================
# Saisonalitätsanalyse über alle Produkte
# ===================================================================

def analysiere_saisonalitaet(df):
    """
    Analysiert und visualisiert die Saisonalität aller Produktgruppen.
    """
    
    print("📅 SAISONALITÄTSANALYSE")
    print("=" * 65)
    
    # Saisonale Indizes berechnen (normalisiert)
    saisonale_indizes = pd.DataFrame(index=range(1, 13))
    
    for produkt in df.columns:
        monatsmittel = df[produkt].groupby(df.index.month).mean()
        gesamtmittel = df[produkt].mean()
        # Saisonaler Index als Prozent vom Durchschnitt
        saisonale_indizes[produkt] = (monatsmittel / gesamtmittel * 100).values
    
    saisonale_indizes.index = ['Jan', 'Feb', 'Mär', 'Apr', 'Mai', 'Jun',
                               'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dez']
    
    # Tabelle ausgeben
    print("\nSaisonale Indizes (100 = Durchschnitt):")
    print("-" * 65)
    print(saisonale_indizes.round(1).to_string())
    
    # Visualisierung
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # 1. Linienplot der saisonalen Indizes
    ax1 = axes[0]
    farben = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D']
    
    for i, produkt in enumerate(saisonale_indizes.columns):
        ax1.plot(range(12), saisonale_indizes[produkt].values, 
                marker='o', linewidth=2, markersize=8,
                label=produkt, color=farben[i])
    
    ax1.axhline(y=100, color='black', linestyle='--', linewidth=1, alpha=0.5)
    ax1.fill_between(range(12), 90, 110, alpha=0.1, color='gray', label='±10% vom Durchschnitt')
    ax1.set_xticks(range(12))
    ax1.set_xticklabels(saisonale_indizes.index)
    ax1.set_xlabel('Monat', fontsize=11)
    ax1.set_ylabel('Saisonaler Index (100 = Durchschnitt)', fontsize=11)
    ax1.set_title('Saisonale Muster nach Produktgruppe', fontsize=12, fontweight='bold')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(80, 130)
    
    # 2. Heatmap
    ax2 = axes[1]
    im = ax2.imshow(saisonale_indizes.T.values, cmap='RdYlGn', aspect='auto',
                   vmin=85, vmax=115)
    
    ax2.set_xticks(range(12))
    ax2.set_xticklabels(saisonale_indizes.index)
    ax2.set_yticks(range(len(df.columns)))
    ax2.set_yticklabels(df.columns)
    ax2.set_xlabel('Monat', fontsize=11)
    ax2.set_title('Saisonalitäts-Heatmap', fontsize=12, fontweight='bold')
    
    # Werte in Zellen
    for i in range(len(df.columns)):
        for j in range(12):
            text = ax2.text(j, i, f'{saisonale_indizes.iloc[j, i]:.0f}',
                           ha='center', va='center', fontsize=9)
    
    plt.colorbar(im, ax=ax2, label='Saisonaler Index')
    
    plt.tight_layout()
    plt.show()
    
    # Erkenntnisse
    print("\n💡 ERKENNTNISSE:")
    for produkt in df.columns:
        max_monat = saisonale_indizes[produkt].idxmax()
        min_monat = saisonale_indizes[produkt].idxmin()
        max_val = saisonale_indizes[produkt].max()
        min_val = saisonale_indizes[produkt].min()
        print(f"   {produkt}:")
        print(f"      Höchste Nachfrage:  {max_monat} (Index: {max_val:.0f})")
        print(f"      Niedrigste Nachfrage: {min_monat} (Index: {min_val:.0f})")
    
    return saisonale_indizes

saisonale_indizes = analysiere_saisonalitaet(nachfrage_df)

---

## 7. Prognose mit externen Faktoren

### Multiple Regression

Wir erweitern unsere Prognose um **externe Faktoren**:

$$\hat{Y}_t = \beta_0 + \beta_1 \cdot Trend_t + \beta_2 \cdot Q4_t + \beta_3 \cdot BlackFriday_t + \epsilon_t$$

| Faktor | Beschreibung |
|--------|-------------|
| **Trend** | Zeitindex (1, 2, 3, ...) |
| **Q4** | Dummy für 4. Quartal (Weihnachtsgeschäft) |
| **BlackFriday** | Dummy für November |

In [ ]:
# ===================================================================
# Prognose mit externen Faktoren
# ===================================================================

def prognose_mit_faktoren(df, produkt, test_perioden=6):
    """
    Erstellt eine Prognose unter Berücksichtigung externer Faktoren.
    
    Parameter:
    ----------
    df : DataFrame
        Nachfragedaten
    produkt : str
        Produktname
    test_perioden : int
        Anzahl Testperioden
        
    Returns:
    --------
    dict : Modellergebnisse
    """
    
    print(f"\n📊 PROGNOSE MIT EXTERNEN FAKTOREN: {produkt}")
    print("=" * 65)
    
    serie = df[produkt]
    
    # Feature-Matrix erstellen
    X = pd.DataFrame(index=serie.index)
    X['Trend'] = np.arange(len(serie))
    X['Q4'] = (serie.index.quarter == 4).astype(int)
    X['November'] = (serie.index.month == 11).astype(int)
    X['Dezember'] = (serie.index.month == 12).astype(int)
    X['Januar'] = (serie.index.month == 1).astype(int)  # Nach-Weihnachts-Tief
    
    # Train-Test Split
    X_train = X.iloc[:-test_perioden]
    X_test = X.iloc[-test_perioden:]
    y_train = serie.iloc[:-test_perioden]
    y_test = serie.iloc[-test_perioden:]
    
    # Modell trainieren
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Prognosen
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Genauigkeit
    genauigkeit_train = berechne_genauigkeit(y_train, y_pred_train)
    genauigkeit_test = berechne_genauigkeit(y_test, y_pred_test)
    
    print("\n📈 MODELLKOEFFIZIENTEN:")
    print("-" * 40)
    print(f"   Intercept (β₀):  {model.intercept_:>10.1f}")
    for feature, coef in zip(X.columns, model.coef_):
        signifikanz = '***' if abs(coef) > 50 else '**' if abs(coef) > 20 else '*' if abs(coef) > 5 else ''
        print(f"   {feature:<12}:     {coef:>10.1f} {signifikanz}")
    
    print("\n📊 GENAUIGKEIT:")
    print("-" * 40)
    print(f"   Training - MAE:  {genauigkeit_train['MAE']:>10.1f}")
    print(f"   Training - MAPE: {genauigkeit_train['MAPE']:>10.1f}%")
    print(f"   Test - MAE:      {genauigkeit_test['MAE']:>10.1f}")
    print(f"   Test - MAPE:     {genauigkeit_test['MAPE']:>10.1f}%")
    print(f"   R² Score:        {model.score(X_test, y_test):>10.3f}")
    
    # Visualisierung
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Tatsächliche Werte
    ax.plot(serie.index, serie.values, 'b-', linewidth=2, 
           label='Tatsächlich', marker='o', markersize=5)
    
    # Prognosen
    ax.plot(y_train.index, y_pred_train, 'g--', linewidth=2, 
           label='Prognose (Training)', alpha=0.7)
    ax.plot(y_test.index, y_pred_test, 'r-', linewidth=3, 
           label=f'Prognose (Test) - MAE: {genauigkeit_test["MAE"]:.0f}', 
           marker='s', markersize=8)
    
    # Vertikale Linie
    ax.axvline(x=y_train.index[-1], color='gray', linestyle='--', linewidth=2, alpha=0.5)
    
    ax.set_title(f'Prognose mit externen Faktoren: {produkt}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Datum', fontsize=11)
    ax.set_ylabel('Nachfrage (Stück)', fontsize=11)
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'model': model,
        'features': list(X.columns),
        'koeffizienten': dict(zip(X.columns, model.coef_)),
        'genauigkeit_test': genauigkeit_test,
        'prognose_test': y_pred_test
    }

# Prognose für Smartphones
ergebnis_faktoren = prognose_mit_faktoren(nachfrage_df, 'Smartphones')

---

## 8. Zukunftsprognose erstellen

Basierend auf unserer Analyse erstellen wir eine **Zukunftsprognose** für die nächsten 6 Monate mit Konfidenzintervall.

In [ ]:
# ===================================================================
# Zukunftsprognose mit Konfidenzintervall
# ===================================================================

def erstelle_zukunftsprognose(df, produkt, horizont=6):
    """
    Erstellt eine Zukunftsprognose mit Konfidenzintervall.
    
    Parameter:
    ----------
    df : DataFrame
        Historische Nachfragedaten
    produkt : str
        Produktname
    horizont : int
        Prognosehorizont in Monaten
        
    Returns:
    --------
    DataFrame : Prognose-Tabelle
    """
    
    print(f"\n🔮 ZUKUNFTSPROGNOSE: {produkt}")
    print("=" * 65)
    
    serie = df[produkt]
    
    # Zukünftige Datumswerte
    letztes_datum = serie.index[-1]
    zukunft_index = pd.date_range(
        start=letztes_datum + pd.DateOffset(months=1),
        periods=horizont,
        freq='M'
    )
    
    # Feature-Matrix für historische + zukünftige Daten
    X_hist = pd.DataFrame(index=serie.index)
    X_hist['Trend'] = np.arange(len(serie))
    X_hist['Q4'] = (serie.index.quarter == 4).astype(int)
    X_hist['November'] = (serie.index.month == 11).astype(int)
    
    X_zukunft = pd.DataFrame(index=zukunft_index)
    X_zukunft['Trend'] = np.arange(len(serie), len(serie) + horizont)
    X_zukunft['Q4'] = (zukunft_index.quarter == 4).astype(int)
    X_zukunft['November'] = (zukunft_index.month == 11).astype(int)
    
    # Modell auf allen historischen Daten trainieren
    model = LinearRegression()
    model.fit(X_hist, serie)
    
    # Prognose erstellen
    prognose = model.predict(X_zukunft)
    
    # Konfidenzintervall (basierend auf historischen Residuen)
    residuen = serie - model.predict(X_hist)
    std_residuen = residuen.std()
    
    # 95% Konfidenzintervall (wird breiter mit Horizont)
    unsicherheit = [std_residuen * 1.96 * np.sqrt(1 + 0.1*i) for i in range(horizont)]
    untere_grenze = prognose - unsicherheit
    obere_grenze = prognose + unsicherheit
    
    # Prognose-Tabelle erstellen
    prognose_df = pd.DataFrame({
        'Datum': zukunft_index,
        'Prognose': np.round(prognose, 0),
        'Untere_Grenze': np.round(untere_grenze, 0),
        'Obere_Grenze': np.round(obere_grenze, 0)
    })
    prognose_df['Monat'] = prognose_df['Datum'].dt.strftime('%Y-%m')
    
    print(f"\n📋 PROGNOSE-TABELLE ({horizont} Monate):")
    print("-" * 65)
    print(f"{'Monat':<10} {'Prognose':>12} {'95%-Konfidenzintervall':>25}")
    print("-" * 50)
    for _, row in prognose_df.iterrows():
        print(f"{row['Monat']:<10} {row['Prognose']:>12.0f} [{row['Untere_Grenze']:>8.0f} - {row['Obere_Grenze']:>8.0f}]")
    
    # Visualisierung
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Historische Daten
    ax.plot(serie.index, serie.values, 'b-', linewidth=2, 
           label='Historisch', marker='o', markersize=4)
    
    # Prognose
    ax.plot(zukunft_index, prognose, 'r-', linewidth=3, 
           label='Prognose', marker='s', markersize=8)
    
    # Konfidenzintervall
    ax.fill_between(zukunft_index, untere_grenze, obere_grenze,
                   alpha=0.3, color='red', label='95%-Konfidenzintervall')
    
    # Vertikale Linie
    ax.axvline(x=serie.index[-1], color='gray', linestyle='--', linewidth=2)
    ax.text(serie.index[-1], ax.get_ylim()[1]*0.95, ' Prognose →', 
           fontsize=11, ha='left', color='gray')
    
    ax.set_title(f'Zukunftsprognose: {produkt} ({horizont} Monate)', 
                fontsize=14, fontweight='bold')
    ax.set_xlabel('Datum', fontsize=11)
    ax.set_ylabel('Nachfrage (Stück)', fontsize=11)
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return prognose_df

# Zukunftsprognose für alle Produkte
print("🔮 ZUKUNFTSPROGNOSEN FÜR ALLE PRODUKTE")
print("=" * 65)

prognosen = {}
for produkt in nachfrage_df.columns:
    prognosen[produkt] = erstelle_zukunftsprognose(nachfrage_df, produkt, horizont=6)

---

## 9. Aufgaben für Studierende

Bearbeiten Sie die folgenden Aufgaben, um Ihr Verständnis der Nachfrageprognose zu vertiefen.

### ✏️ Aufgabe 1: Parameter-Optimierung für Exponentielles Glätten

Der Glättungsparameter α beeinflusst die Prognosegenauigkeit:
- **Kleines α (z.B. 0.1)**: Langsame Anpassung, glättet stark
- **Großes α (z.B. 0.9)**: Schnelle Anpassung, reagiert auf Änderungen

**Aufgaben:**
1. Testen Sie α-Werte von 0.1 bis 0.9 für das Produkt "Tablets"
2. Welcher Wert minimiert den MAE auf den Testdaten?
3. Visualisieren Sie den Zusammenhang zwischen α und MAE

In [ ]:
# Ihre Lösung für Aufgabe 1:
# ===========================

# TODO: Implementieren Sie eine Schleife über verschiedene Alpha-Werte
# TODO: Berechnen Sie jeweils den MAE auf den Testdaten
# TODO: Finden Sie das optimale Alpha
# TODO: Visualisieren Sie die Ergebnisse



In [ ]:
# ===================================================================
# LÖSUNG Aufgabe 1 (zur Selbstkontrolle)
# ===================================================================

print("📝 LÖSUNG AUFGABE 1: Parameter-Optimierung")
print("=" * 65)

serie = nachfrage_df['Tablets']
test_perioden = 6
train = serie[:-test_perioden]
test = serie[-test_perioden:]

# Alpha-Werte testen
alpha_werte = np.arange(0.1, 1.0, 0.1)
ergebnisse = []

print(f"\n{'Alpha':<10} {'MAE':>10} {'RMSE':>10} {'MAPE (%)':>10}")
print("-" * 45)

for alpha in alpha_werte:
    prognose = prog.exponential_smoothing(train, alpha, test_perioden)
    genauigkeit = berechne_genauigkeit(test.values, prognose)
    ergebnisse.append({
        'alpha': alpha,
        'MAE': genauigkeit['MAE'],
        'RMSE': genauigkeit['RMSE'],
        'MAPE': genauigkeit['MAPE']
    })
    print(f"{alpha:<10.1f} {genauigkeit['MAE']:>10.1f} {genauigkeit['RMSE']:>10.1f} {genauigkeit['MAPE']:>10.1f}")

df_alpha = pd.DataFrame(ergebnisse)
optimales_alpha = df_alpha.loc[df_alpha['MAE'].idxmin(), 'alpha']
bester_mae = df_alpha['MAE'].min()

print("-" * 45)
print(f"\n🏆 Optimales Alpha: {optimales_alpha:.1f} (MAE: {bester_mae:.1f})")

# Visualisierung
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(df_alpha['alpha'], df_alpha['MAE'], 'b-o', linewidth=2, markersize=10, label='MAE')
ax.axvline(x=optimales_alpha, color='red', linestyle='--', linewidth=2, 
          label=f'Optimum: α = {optimales_alpha:.1f}')

ax.set_xlabel('Alpha (Glättungsparameter)', fontsize=11)
ax.set_ylabel('MAE', fontsize=11)
ax.set_title('Optimierung des Glättungsparameters α', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 INTERPRETATION:")
print(f"   • Das optimale Alpha liegt bei {optimales_alpha:.1f}")
if optimales_alpha > 0.5:
    print("   • Ein hoher Wert deutet auf schnelle Änderungen in den Daten hin")
else:
    print("   • Ein niedriger Wert deutet auf stabile, trendige Daten hin")

### ✏️ Aufgabe 2: Vergleich aller Produkte

Führen Sie den Prognosevergleich für **alle Produktgruppen** durch und beantworten Sie:

1. Welche Methode funktioniert am besten für jedes Produkt?
2. Gibt es Unterschiede in der Prognostizierbarkeit zwischen den Produkten?
3. Welches Produkt ist am schwierigsten zu prognostizieren? Warum?

In [ ]:
# Ihre Lösung für Aufgabe 2:
# ===========================

# TODO: Führen Sie den Prognosevergleich für alle Produkte durch
# TODO: Erstellen Sie eine Zusammenfassung
# TODO: Analysieren Sie die Unterschiede



In [ ]:
# ===================================================================
# LÖSUNG Aufgabe 2 (zur Selbstkontrolle)
# ===================================================================

print("📝 LÖSUNG AUFGABE 2: Vergleich aller Produkte")
print("=" * 70)

zusammenfassung = []

for produkt in nachfrage_df.columns:
    serie = nachfrage_df[produkt]
    test_perioden = 6
    train = serie[:-test_perioden]
    test = serie[-test_perioden:]
    
    # Alle Methoden testen
    methoden_ergebnisse = {
        'Naive': prog.naive(train, test_perioden),
        'Saisonale Naive': prog.saisonale_naive(train, 12, test_perioden),
        'Moving Avg (3)': prog.moving_average(train, 3, test_perioden),
        'Exp. Smoothing': prog.exponential_smoothing(train, 0.3, test_perioden),
        'Holt Linear': prog.holt_linear(train, 0.3, 0.1, test_perioden),
        'Lin. Regression': prog.linear_regression(train, test_perioden)
    }
    
    beste_methode = None
    bester_mae = float('inf')
    
    for methode, prognose in methoden_ergebnisse.items():
        mae = mean_absolute_error(test.values, prognose)
        if mae < bester_mae:
            bester_mae = mae
            beste_methode = methode
    
    mape = np.mean(np.abs((test.values - methoden_ergebnisse[beste_methode]) / test.values)) * 100
    
    zusammenfassung.append({
        'Produkt': produkt,
        'Beste Methode': beste_methode,
        'MAE': bester_mae,
        'MAPE (%)': mape,
        'Durchschn. Nachfrage': serie.mean()
    })

df_zusammenfassung = pd.DataFrame(zusammenfassung)
df_zusammenfassung['Rel. Fehler (%)'] = df_zusammenfassung['MAE'] / df_zusammenfassung['Durchschn. Nachfrage'] * 100

print("\n📊 ZUSAMMENFASSUNG:")
print("-" * 80)
print(df_zusammenfassung.to_string(index=False))

# Schwierigste Prognose identifizieren
schwiergstes = df_zusammenfassung.loc[df_zusammenfassung['MAPE (%)'].idxmax()]
einfachstes = df_zusammenfassung.loc[df_zusammenfassung['MAPE (%)'].idxmin()]

print("\n💡 ERKENNTNISSE:")
print(f"   • Am schwierigsten zu prognostizieren: {schwiergstes['Produkt']} (MAPE: {schwiergstes['MAPE (%)']:.1f}%)")
print(f"   • Am einfachsten zu prognostizieren: {einfachstes['Produkt']} (MAPE: {einfachstes['MAPE (%)']:.1f}%)")
print("\n   Mögliche Gründe für Unterschiede:")
print("   • Höhere Variabilität/Rauschen in den Daten")
print("   • Stärkere oder unregelmäßigere Saisonalität")
print("   • Externe Einflüsse (Wettbewerb, Trends)")

### ✏️ Aufgabe 3: Kombinierte Prognose

Oft liefert eine **Kombination** mehrerer Methoden bessere Ergebnisse als einzelne Methoden.

**Aufgaben:**
1. Erstellen Sie eine kombinierte Prognose als gewichteten Durchschnitt von 3 Methoden
2. Testen Sie verschiedene Gewichtungen
3. Vergleichen Sie die kombinierte Prognose mit den Einzelmethoden

In [ ]:
# Ihre Lösung für Aufgabe 3:
# ===========================

# TODO: Erstellen Sie Prognosen mit 3 verschiedenen Methoden
# TODO: Kombinieren Sie diese mit verschiedenen Gewichtungen
# TODO: Vergleichen Sie die Genauigkeit



In [ ]:
# ===================================================================
# LÖSUNG Aufgabe 3 (zur Selbstkontrolle)
# ===================================================================

print("📝 LÖSUNG AUFGABE 3: Kombinierte Prognose")
print("=" * 65)

serie = nachfrage_df['Smartphones']
test_perioden = 6
train = serie[:-test_perioden]
test = serie[-test_perioden:]

# Einzelprognosen erstellen
prognose_ma = np.array(prog.moving_average(train, 3, test_perioden))
prognose_exp = np.array(prog.exponential_smoothing(train, 0.3, test_perioden))
prognose_holt = np.array(prog.holt_linear(train, 0.3, 0.1, test_perioden))

print("\n1️⃣ Einzelprognosen:")
print("-" * 50)

einzelmethoden = {
    'Moving Average': prognose_ma,
    'Exp. Smoothing': prognose_exp,
    'Holt Linear': prognose_holt
}

for name, prognose in einzelmethoden.items():
    mae = mean_absolute_error(test.values, prognose)
    print(f"   {name:<20}: MAE = {mae:.1f}")

print("\n2️⃣ Kombinierte Prognosen (verschiedene Gewichtungen):")
print("-" * 50)

# Verschiedene Gewichtungen testen
gewichtungen = [
    {'name': 'Gleichgewichtet', 'w': [1/3, 1/3, 1/3]},
    {'name': 'Holt-fokussiert', 'w': [0.2, 0.2, 0.6]},
    {'name': 'Exp-fokussiert', 'w': [0.2, 0.6, 0.2]},
    {'name': 'MA-fokussiert', 'w': [0.6, 0.2, 0.2]},
]

beste_kombination = None
bester_mae = float('inf')

for gew in gewichtungen:
    w = gew['w']
    kombiniert = w[0] * prognose_ma + w[1] * prognose_exp + w[2] * prognose_holt
    mae = mean_absolute_error(test.values, kombiniert)
    
    print(f"   {gew['name']:<20} (w={w}): MAE = {mae:.1f}")
    
    if mae < bester_mae:
        bester_mae = mae
        beste_kombination = gew['name']
        beste_prognose = kombiniert

print("-" * 50)
print(f"\n🏆 Beste Kombination: {beste_kombination} (MAE: {bester_mae:.1f})")

# Visualisierung
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(train.index, train.values, 'b-', linewidth=1.5, label='Training', alpha=0.7)
ax.plot(test.index, test.values, 'go-', linewidth=3, markersize=10, label='Tatsächlich')

ax.plot(test.index, prognose_ma, '--', color='orange', label=f'Moving Avg', linewidth=2)
ax.plot(test.index, prognose_exp, '--', color='purple', label=f'Exp. Smoothing', linewidth=2)
ax.plot(test.index, prognose_holt, '--', color='brown', label=f'Holt', linewidth=2)
ax.plot(test.index, beste_prognose, 'r-', linewidth=3, marker='s', markersize=8,
       label=f'Kombiniert ({beste_kombination}) - MAE: {bester_mae:.0f}')

ax.set_title('Kombinierte Prognose vs. Einzelmethoden', fontsize=12, fontweight='bold')
ax.set_xlabel('Datum', fontsize=11)
ax.set_ylabel('Nachfrage (Stück)', fontsize=11)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)
ax.axvline(x=train.index[-1], color='gray', linestyle='--', linewidth=2, alpha=0.5)

plt.tight_layout()
plt.show()

print("\n💡 ERKENNTNIS:")
print("   Kombinierte Prognosen können die Stärken verschiedener Methoden")
print("   vereinen und sind oft robuster gegenüber einzelnen Ausreißern.")

---

## 10. Zusammenfassung und Ausblick

### Was haben wir gelernt?

| Thema | Kernaussage |
|-------|-------------|
| **Zeitreihenzerlegung** | Jede Zeitreihe besteht aus Trend, Saisonalität und Rauschen |
| **Prognosemethoden** | Von einfach (Naive) bis komplex (Holt-Winters) - keine universell beste Methode |
| **Genauigkeitsmaße** | MAE, RMSE, MAPE - verschiedene Perspektiven auf den Fehler |
| **Saisonalität** | Kritisch für Produktionsplanung - November (Black Friday) besonders wichtig |
| **Externe Faktoren** | Verbessern die Prognose signifikant |
| **Kombinierte Prognosen** | Oft besser als Einzelmethoden |

### Praktische Empfehlungen

1. **Starten Sie einfach**: Naive/Saisonale Naive als Benchmark
2. **Verstehen Sie Ihre Daten**: Zeitreihenzerlegung vor Prognose
3. **Testen Sie mehrere Methoden**: Keine Methode ist immer optimal
4. **Berücksichtigen Sie externe Faktoren**: Promotions, Feiertage, Wettbewerb
5. **Validieren Sie regelmäßig**: Modelle können sich verschlechtern

### Grenzen und Weiterführendes

- ❌ Klassische Methoden können keine komplexen Muster lernen
- ❌ Keine automatische Anpassung an Strukturbrüche

**Weiterführende Themen:**
- **ARIMA-Modelle**: Autoregressive integrierte Moving Average
- **Machine Learning**: Random Forest, Gradient Boosting für Zeitreihen
- **Deep Learning**: LSTM, Transformer für komplexe Muster
- **Hierarchische Prognosen**: Aggregation über Produkthierarchien

In [ ]:
# ===================================================================
# Abschluss und Checkliste
# ===================================================================

print("\n" + "="*70)
print("📋 CHECKLISTE - Data Analytics & Prognose")
print("="*70)

checkliste = [
    ("Zeitreihe visualisieren und verstehen", 
     "Trend, Saisonalität, Ausreißer identifizieren"),
    ("Zeitreihenzerlegung durchführen", 
     "Trend, Saisonalität, Rest trennen"),
    ("Prognosemethoden anwenden", 
     "Naive, MA, Exp. Smoothing, Holt, Regression"),
    ("Train-Test-Split durchführen", 
     "Letzte Perioden für Validierung zurückhalten"),
    ("Genauigkeit messen", 
     "MAE, RMSE, MAPE berechnen und interpretieren"),
    ("Saisonalität analysieren", 
     "Saisonale Indizes berechnen, Hoch-/Tiefpunkte identifizieren"),
    ("Externe Faktoren einbeziehen", 
     "Promotions, Feiertage, Wetterdaten etc."),
    ("Zukunftsprognose erstellen", 
     "Mit Konfidenzintervall für Unsicherheit")
]

for i, (punkt, beschreibung) in enumerate(checkliste, 1):
    print(f"\n☐ {i}. {punkt}")
    print(f"   → {beschreibung}")

print("\n" + "="*70)
print("🎓 Viel Erfolg bei der Anwendung in der Praxis!")
print("   Nächstes Thema: Machine Learning in der Produktionsplanung")
print("="*70)